In [1]:
import numpy as np, os

# paths
seq_label_dir = "dataset/referkitti/KITTI/training/labels_with_ids/image_02/0001"
gt_seq_path   = "outputs/referkitti_rmot_2025-11-14_1213/gt_seq/0001.txt"

print("First few raw label files in", seq_label_dir)
for fn in sorted(os.listdir(seq_label_dir))[:3]:
    print("===", fn, "===")
    with open(os.path.join(seq_label_dir, fn)) as f:
        for i, line in enumerate(f):
            if i == 5: break
            print("  ", line.rstrip())
    print()

print("\nFirst 10 rows of GT seq file:")
gt = np.loadtxt(gt_seq_path, delimiter=",")
print(gt[:10])


First few raw label files in dataset/referkitti/KITTI/training/labels_with_ids/image_02/0001
=== 000000.txt ===
   0 11 0.634461 0.493333 0.354267 0.506667
   0 12 0.578905 0.488000 0.099034 0.242667
   0 13 0.553140 0.482667 0.046699 0.160000
   0 14 0.310789 0.520000 0.061192 0.130667
   0 2 0.400966 0.504000 0.023349 0.061333

=== 000001.txt ===
   0 11 0.657005 0.498667 0.342995 0.501333
   0 12 0.586151 0.488000 0.116747 0.269333
   0 13 0.555556 0.485333 0.051530 0.170667
   0 14 0.302738 0.525333 0.063607 0.141333
   0 2 0.398551 0.509333 0.024960 0.061333

=== 000002.txt ===
   0 11 0.685990 0.506667 0.314010 0.493333
   0 12 0.593398 0.490667 0.139291 0.298667
   0 13 0.560386 0.485333 0.057166 0.184000
   0 14 0.290660 0.520000 0.069243 0.152000
   0 2 0.395330 0.504000 0.026570 0.064000


First 10 rows of GT seq file:
[[  0.   11.  568.   90.  440.  190.    1.   -1.   -1.   -1. ]
 [  0.   12.  657.5 137.5 123.   91.    1.   -1.   -1.   -1. ]
 [  0.   13.  658.  151.   58.   

In [3]:
# ================================
# ORACLE CHECK FOR REFER-KITTI RMOT
# ================================
import os
import pandas as pd
from eval.compute_metrics import MotMetricsEvaluator

# --- 1. Choose your run directory ---
RUN_DIR = "outputs/referkitti_rmot_2025-11-14_1236"   # <-- CHANGE IF NEEDED

gt_expr_dir = os.path.join(RUN_DIR, "gt_expr")
res_expr_dir = os.path.join(RUN_DIR, "results_expr")

print("GT expr dir:", gt_expr_dir)
print("RES expr dir:", res_expr_dir)

# --- 2. List available expression files ---
gt_files = sorted(os.listdir(gt_expr_dir))
print("\nAvailable expressions:")
for i, f in enumerate(gt_files):
    print(f"{i:03d} → {f}")

# --- 3. Pick an expression index ---
expr_idx = 0   # <---- CHANGE THIS TO CHOOSE WHICH EXPRESSION TO ORACLE
expr_file = gt_files[expr_idx]

print(f"\nUsing expression: {expr_file}")

gt_path = os.path.join(gt_expr_dir, expr_file)
res_path = os.path.join(res_expr_dir, expr_file)

# --- 4. ORACLE: copy GT to results ---
os.makedirs(res_expr_dir, exist_ok=True)
!cp "{gt_path}" "{res_path}"

print("\n✓ Oracle result created:")
print(res_path)

# --- 5. Evaluate ONLY these files ---
evaluator = MotMetricsEvaluator(distth=0.5, fmt="mot15-2D")
df = evaluator.evaluate(gt_expr_dir, res_expr_dir, verbose=True)

# --- 6. Show summary ---
if df is not None:
    if "AVG" in df.index:
        mota = float(df.loc["AVG"].get("mota", float("nan")))
        idf1 = float(df.loc["AVG"].get("idf1", float("nan")))
    else:
        mota = df["mota"].mean()
        idf1 = df["idf1"].mean()

    print("\n================ ORACLE RMOT SUMMARY ================")
    print(f"MOTA = {mota*100:.2f}%")
    print(f"IDF1 = {idf1*100:.2f}%")
    print("=====================================================")
else:
    print("⚠ No matching GT/RES files!")


GT expr dir: outputs/referkitti_rmot_2025-11-14_1236/gt_expr
RES expr dir: outputs/referkitti_rmot_2025-11-14_1236/results_expr

Available expressions:
000 → 0001_expr0000.txt
001 → 0001_expr0001.txt

Using expression: 0001_expr0000.txt

✓ Oracle result created:
outputs/referkitti_rmot_2025-11-14_1236/results_expr/0001_expr0000.txt
📂 Found 2 GT files and 2 result files

===== 0001_expr0000 =====
              num_frames   MOTA  MOTP   IDF1    IDP    IDR   Prcn   Rcll IDs MT ML PT  FM FP FN num_objects num_matches
0001_expr0000        228 100.0% 0.000 100.0% 100.0% 100.0% 100.0% 100.0%   0 13  0  0   0  0  0         385         385

===== 0001_expr0001 =====
              num_frames    MOTA MOTP IDF1  IDP  IDR Prcn Rcll IDs MT ML PT  FM   FP  FN num_objects num_matches
0001_expr0001        426 -381.9%  NaN 0.0% 0.0% 0.0% 0.0% 0.0%   0  0 18  0   0 2303 603         603           0

====== AVERAGE ======
               num_frames      mota  motp  idf1  idp  idr  precision  recall  \
0001_